In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

In [3]:
# Convolutions
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# Pooling
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [4]:
# model
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# session
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 35095.1875 Validation Accuracy: 0.171875
Epoch  1, Batch   2 -Loss: 29733.6758 Validation Accuracy: 0.171875
Epoch  1, Batch   3 -Loss: 25757.6738 Validation Accuracy: 0.171875
Epoch  1, Batch   4 -Loss: 26477.0430 Validation Accuracy: 0.160156
Epoch  1, Batch   5 -Loss: 21802.0137 Validation Accuracy: 0.175781
Epoch  1, Batch   6 -Loss: 20907.3965 Validation Accuracy: 0.191406
Epoch  1, Batch   7 -Loss: 18713.2539 Validation Accuracy: 0.191406
Epoch  1, Batch   8 -Loss: 20335.6426 Validation Accuracy: 0.191406
Epoch  1, Batch   9 -Loss: 18040.1289 Validation Accuracy: 0.195312
Epoch  1, Batch  10 -Loss: 19019.4043 Validation Accuracy: 0.210938
Epoch  1, Batch  11 -Loss: 17492.8926 Validation Accuracy: 0.195312
Epoch  1, Batch  12 -Loss: 14294.5684 Validation Accuracy: 0.203125
Epoch  1, Batch  13 -Loss: 16498.0547 Validation Accuracy: 0.214844
Epoch  1, Batch  14 -Loss: 20589.6094 Validation Accuracy: 0.222656
Epoch  1, Batch  15 -Loss: 15189.4961 Validation

Epoch  1, Batch 122 -Loss:  4674.2178 Validation Accuracy: 0.531250
Epoch  1, Batch 123 -Loss:  4181.0977 Validation Accuracy: 0.546875
Epoch  1, Batch 124 -Loss:  4955.6616 Validation Accuracy: 0.546875
Epoch  1, Batch 125 -Loss:  4749.5132 Validation Accuracy: 0.539062
Epoch  1, Batch 126 -Loss:  4017.0142 Validation Accuracy: 0.558594
Epoch  1, Batch 127 -Loss:  4007.4585 Validation Accuracy: 0.558594
Epoch  1, Batch 128 -Loss:  4670.9775 Validation Accuracy: 0.554688
Epoch  1, Batch 129 -Loss:  4230.3555 Validation Accuracy: 0.562500
Epoch  1, Batch 130 -Loss:  3900.8755 Validation Accuracy: 0.558594
Epoch  1, Batch 131 -Loss:  3991.7788 Validation Accuracy: 0.554688
Epoch  1, Batch 132 -Loss:  4413.3008 Validation Accuracy: 0.562500
Epoch  1, Batch 133 -Loss:  3535.5837 Validation Accuracy: 0.566406
Epoch  1, Batch 134 -Loss:  3437.0537 Validation Accuracy: 0.566406
Epoch  1, Batch 135 -Loss:  4827.4307 Validation Accuracy: 0.570312
Epoch  1, Batch 136 -Loss:  3829.3354 Validation

Epoch  1, Batch 243 -Loss:  2214.5515 Validation Accuracy: 0.644531
Epoch  1, Batch 244 -Loss:  2545.9768 Validation Accuracy: 0.632812
Epoch  1, Batch 245 -Loss:  2463.0334 Validation Accuracy: 0.636719
Epoch  1, Batch 246 -Loss:  2261.5596 Validation Accuracy: 0.660156
Epoch  1, Batch 247 -Loss:  2840.5322 Validation Accuracy: 0.652344
Epoch  1, Batch 248 -Loss:  3680.1055 Validation Accuracy: 0.640625
Epoch  1, Batch 249 -Loss:  1668.8723 Validation Accuracy: 0.648438
Epoch  1, Batch 250 -Loss:  2062.7046 Validation Accuracy: 0.667969
Epoch  1, Batch 251 -Loss:  2429.7290 Validation Accuracy: 0.656250
Epoch  1, Batch 252 -Loss:  2624.4319 Validation Accuracy: 0.664062
Epoch  1, Batch 253 -Loss:  2716.8486 Validation Accuracy: 0.652344
Epoch  1, Batch 254 -Loss:  2356.6030 Validation Accuracy: 0.648438
Epoch  1, Batch 255 -Loss:  1939.1018 Validation Accuracy: 0.664062
Epoch  1, Batch 256 -Loss:  2296.7581 Validation Accuracy: 0.656250
Epoch  1, Batch 257 -Loss:  1697.2972 Validation

Epoch  1, Batch 364 -Loss:  2149.4194 Validation Accuracy: 0.695312
Epoch  1, Batch 365 -Loss:  1154.9932 Validation Accuracy: 0.699219
Epoch  1, Batch 366 -Loss:  1609.3167 Validation Accuracy: 0.695312
Epoch  1, Batch 367 -Loss:  2000.3901 Validation Accuracy: 0.695312
Epoch  1, Batch 368 -Loss:  1364.7056 Validation Accuracy: 0.699219
Epoch  1, Batch 369 -Loss:  1432.1543 Validation Accuracy: 0.699219
Epoch  1, Batch 370 -Loss:  1283.9094 Validation Accuracy: 0.699219
Epoch  1, Batch 371 -Loss:  1244.3041 Validation Accuracy: 0.695312
Epoch  1, Batch 372 -Loss:  1329.6010 Validation Accuracy: 0.699219
Epoch  1, Batch 373 -Loss:  1767.3818 Validation Accuracy: 0.703125
Epoch  1, Batch 374 -Loss:  1555.0178 Validation Accuracy: 0.699219
Epoch  1, Batch 375 -Loss:  1244.0061 Validation Accuracy: 0.695312
Epoch  1, Batch 376 -Loss:  1048.9644 Validation Accuracy: 0.691406
Epoch  1, Batch 377 -Loss:  2273.0232 Validation Accuracy: 0.691406
Epoch  1, Batch 378 -Loss:  1403.4385 Validation

Epoch  2, Batch  56 -Loss:  1273.4681 Validation Accuracy: 0.714844
Epoch  2, Batch  57 -Loss:  1252.9233 Validation Accuracy: 0.707031
Epoch  2, Batch  58 -Loss:  1272.6920 Validation Accuracy: 0.714844
Epoch  2, Batch  59 -Loss:  1456.9639 Validation Accuracy: 0.714844
Epoch  2, Batch  60 -Loss:  1030.4216 Validation Accuracy: 0.726562
Epoch  2, Batch  61 -Loss:  1088.3828 Validation Accuracy: 0.722656
Epoch  2, Batch  62 -Loss:  1733.2942 Validation Accuracy: 0.722656
Epoch  2, Batch  63 -Loss:  1081.2583 Validation Accuracy: 0.730469
Epoch  2, Batch  64 -Loss:  1820.6515 Validation Accuracy: 0.726562
Epoch  2, Batch  65 -Loss:  1624.3252 Validation Accuracy: 0.730469
Epoch  2, Batch  66 -Loss:  1887.3442 Validation Accuracy: 0.730469
Epoch  2, Batch  67 -Loss:  1358.3457 Validation Accuracy: 0.726562
Epoch  2, Batch  68 -Loss:  1257.7544 Validation Accuracy: 0.722656
Epoch  2, Batch  69 -Loss:  1674.6631 Validation Accuracy: 0.718750
Epoch  2, Batch  70 -Loss:  1182.9713 Validation

...

Epoch  9, Batch 320 -Loss:   320.9636 Validation Accuracy: 0.828125
Epoch  9, Batch 321 -Loss:   321.6334 Validation Accuracy: 0.828125
Epoch  9, Batch 322 -Loss:   355.8013 Validation Accuracy: 0.828125
Epoch  9, Batch 323 -Loss:   262.4027 Validation Accuracy: 0.828125
Epoch  9, Batch 324 -Loss:   197.5901 Validation Accuracy: 0.828125
Epoch  9, Batch 325 -Loss:   194.7861 Validation Accuracy: 0.832031
Epoch  9, Batch 326 -Loss:   205.4671 Validation Accuracy: 0.832031
Epoch  9, Batch 327 -Loss:   355.8050 Validation Accuracy: 0.832031
Epoch  9, Batch 328 -Loss:   241.1269 Validation Accuracy: 0.832031
Epoch  9, Batch 329 -Loss:   311.6750 Validation Accuracy: 0.832031
Epoch  9, Batch 330 -Loss:   158.3788 Validation Accuracy: 0.832031
Epoch  9, Batch 331 -Loss:   175.1577 Validation Accuracy: 0.832031
Epoch  9, Batch 332 -Loss:   318.7785 Validation Accuracy: 0.832031
Epoch  9, Batch 333 -Loss:   210.5059 Validation Accuracy: 0.832031
Epoch  9, Batch 334 -Loss:   334.2498 Validation

Epoch 10, Batch  12 -Loss:   295.1715 Validation Accuracy: 0.824219
Epoch 10, Batch  13 -Loss:   314.6563 Validation Accuracy: 0.816406
Epoch 10, Batch  14 -Loss:   283.9154 Validation Accuracy: 0.820312
Epoch 10, Batch  15 -Loss:   207.1831 Validation Accuracy: 0.824219
Epoch 10, Batch  16 -Loss:   369.6870 Validation Accuracy: 0.832031
Epoch 10, Batch  17 -Loss:   306.6342 Validation Accuracy: 0.828125
Epoch 10, Batch  18 -Loss:   326.4649 Validation Accuracy: 0.832031
Epoch 10, Batch  19 -Loss:   346.2832 Validation Accuracy: 0.828125
Epoch 10, Batch  20 -Loss:   316.3344 Validation Accuracy: 0.828125
Epoch 10, Batch  21 -Loss:   245.5986 Validation Accuracy: 0.824219
Epoch 10, Batch  22 -Loss:   367.5427 Validation Accuracy: 0.824219
Epoch 10, Batch  23 -Loss:   328.2094 Validation Accuracy: 0.824219
Epoch 10, Batch  24 -Loss:   305.4090 Validation Accuracy: 0.824219
Epoch 10, Batch  25 -Loss:   320.8890 Validation Accuracy: 0.824219
Epoch 10, Batch  26 -Loss:   182.4964 Validation

Epoch 10, Batch 133 -Loss:   416.0352 Validation Accuracy: 0.816406
Epoch 10, Batch 134 -Loss:   317.0324 Validation Accuracy: 0.816406
Epoch 10, Batch 135 -Loss:   267.5917 Validation Accuracy: 0.816406
Epoch 10, Batch 136 -Loss:   326.7870 Validation Accuracy: 0.816406
Epoch 10, Batch 137 -Loss:   375.8657 Validation Accuracy: 0.812500
Epoch 10, Batch 138 -Loss:   274.0400 Validation Accuracy: 0.812500
Epoch 10, Batch 139 -Loss:   300.1685 Validation Accuracy: 0.812500
Epoch 10, Batch 140 -Loss:   285.7752 Validation Accuracy: 0.812500
Epoch 10, Batch 141 -Loss:   189.5892 Validation Accuracy: 0.812500
Epoch 10, Batch 142 -Loss:   211.8315 Validation Accuracy: 0.812500
Epoch 10, Batch 143 -Loss:   329.9581 Validation Accuracy: 0.812500
Epoch 10, Batch 144 -Loss:   131.3235 Validation Accuracy: 0.812500
Epoch 10, Batch 145 -Loss:   288.4373 Validation Accuracy: 0.812500
Epoch 10, Batch 146 -Loss:   244.8050 Validation Accuracy: 0.812500
Epoch 10, Batch 147 -Loss:   141.9818 Validation

Epoch 10, Batch 254 -Loss:   170.5187 Validation Accuracy: 0.812500
Epoch 10, Batch 255 -Loss:   213.8324 Validation Accuracy: 0.808594
Epoch 10, Batch 256 -Loss:   273.4249 Validation Accuracy: 0.812500
Epoch 10, Batch 257 -Loss:   222.7009 Validation Accuracy: 0.812500
Epoch 10, Batch 258 -Loss:   279.5208 Validation Accuracy: 0.812500
Epoch 10, Batch 259 -Loss:   242.6709 Validation Accuracy: 0.812500
Epoch 10, Batch 260 -Loss:   226.2928 Validation Accuracy: 0.812500
Epoch 10, Batch 261 -Loss:   293.6483 Validation Accuracy: 0.816406
Epoch 10, Batch 262 -Loss:   335.7797 Validation Accuracy: 0.816406
Epoch 10, Batch 263 -Loss:   269.5733 Validation Accuracy: 0.820312
Epoch 10, Batch 264 -Loss:   217.8257 Validation Accuracy: 0.816406
Epoch 10, Batch 265 -Loss:   262.7198 Validation Accuracy: 0.820312
Epoch 10, Batch 266 -Loss:   195.4446 Validation Accuracy: 0.812500
Epoch 10, Batch 267 -Loss:   240.8168 Validation Accuracy: 0.816406
Epoch 10, Batch 268 -Loss:   138.1356 Validation

Epoch 10, Batch 375 -Loss:   152.6900 Validation Accuracy: 0.808594
Epoch 10, Batch 376 -Loss:   213.2320 Validation Accuracy: 0.808594
Epoch 10, Batch 377 -Loss:   307.7869 Validation Accuracy: 0.808594
Epoch 10, Batch 378 -Loss:   302.6185 Validation Accuracy: 0.808594
Epoch 10, Batch 379 -Loss:   146.6325 Validation Accuracy: 0.808594
Epoch 10, Batch 380 -Loss:   284.8491 Validation Accuracy: 0.812500
Epoch 10, Batch 381 -Loss:   385.8154 Validation Accuracy: 0.812500
Epoch 10, Batch 382 -Loss:   200.7428 Validation Accuracy: 0.812500
Epoch 10, Batch 383 -Loss:   288.0809 Validation Accuracy: 0.812500
Epoch 10, Batch 384 -Loss:   255.1970 Validation Accuracy: 0.808594
Epoch 10, Batch 385 -Loss:   297.7589 Validation Accuracy: 0.808594
Epoch 10, Batch 386 -Loss:   227.6932 Validation Accuracy: 0.800781
Epoch 10, Batch 387 -Loss:   376.7026 Validation Accuracy: 0.800781
Epoch 10, Batch 388 -Loss:   290.3044 Validation Accuracy: 0.808594
Epoch 10, Batch 389 -Loss:   303.7422 Validation